## PREPROCESAMIENTO DE DATOS

In [2]:
import pandas as pd

train = pd.read_csv("C:\\Users\\USUARIO\\competicion_ml_ia\\p3\\data\\public_train.csv")
train.head()


,ccs,adduct,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,desc_8,...,fgp_611,fgp_612,fgp_613,fgp_614,fgp_615,fgp_616,fgp_617,fgp_618,fgp_619,fgp_620
0,155.71001,Monomer_[M+H],-5.27492,420.85751,0.79886,60.68518,0.22702,54.36304,55.73885,8.28095,...,0,0,1,0,1,1,1,0,1,0
1,179.56000,Monomer_[M-H],-2.00000,1171.97156,1.13333,121.96276,0.15520,100.45390,97.68512,11.36667,...,1,0,0,1,0,1,0,1,1,0
2,180.00000,Monomer_[M+Na],-0.70060,1447.22644,1.30616,141.10509,0.13916,110.47855,108.96603,12.96667,...,0,1,1,0,1,0,0,0,0,1
3,155.53999,Monomer_[M+Na],0.00000,479.80600,0.92774,74.90678,0.21166,65.64590,66.68343,6.83333,...,1,0,1,1,1,0,1,1,0,0
4,173.50000,Monomer_[M+H],-1.63669,978.85028,1.24843,112.27611,0.15789,87.88085,89.68015,10.73333,...,0,1,0,1,0,0,1,0,1,0


In [3]:
test = pd.read_csv("C:\\Users\\USUARIO\\competicion_ml_ia\\p3\\data\\public_test.csv")
test.head()

,adduct,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,desc_8,desc_9,...,fgp_611,fgp_612,fgp_613,fgp_614,fgp_615,fgp_616,fgp_617,fgp_618,fgp_619,fgp_620
0,Monomer_[M+H],-2.52359,942.17682,1.29512,87.64198,0.16297,83.92964,84.34066,11.00000,4.80405,...,0,1,0,1,1,1,1,1,0,1
1,Monomer_[M-H],2.00538,12122.62988,1.80601,621.73560,0.07288,545.80798,554.61530,22.73333,5.64051,...,0,0,0,1,0,1,1,1,1,1
2,Monomer_[M-H],2.73984,6543.37451,1.52727,440.95099,0.08565,305.62857,313.87540,19.90000,5.31395,...,1,1,1,0,0,0,0,0,1,0
3,Monomer_[M+Na],-2.34683,695.94104,1.04344,74.62116,0.17652,65.28460,63.78675,10.81429,4.25346,...,1,0,0,1,1,0,0,0,1,1
4,Monomer_[M+H],0.00000,101.73023,0.03813,20.08101,0.35359,19.35335,20.59024,6.06667,2.62874,...,1,1,0,0,1,1,0,1,1,1


Ya visaulizados los datos, se procede a realizar el preprocesamiento de los mismos. Para ello, se realizarán las siguientes tareas:
- Eliminación de columnas innecesarias(Ingeniería de características)
- Immputación
- Codificación de variables categóricas
- Normalización de los datos


In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
#Codificación de la columna adduct: Convierte la columna categórica adduct en variables numéricas usando técnicas como One-Hot Encoding o Label Encoding.
encoder = OneHotEncoder(sparse_output = False)
adduct_encoded = encoder.fit_transform(train[['adduct']])
train_encoded = pd.concat([train.drop(columns=['adduct']), pd.DataFrame(adduct_encoded)], axis=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(train_encoded.drop(columns=['ccs']).values)


## ENTRENAMIENTO Y ESTIMACIÓN DE ERROR

Después de realizar el preprocesamiento de los datos, se procederá a entrenar un modelo de regresión logística y a estimar el error del modelo. Para ello, se realizarán las siguientes tareas:
- División de los datos en conjuntos de entrenamiento y prueba
- Entrenamiento del modelo
- Estimación del error del modelo

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression
import pandas as pd

# Asegúrate de que los nombres de las columnas sean cadenas
X = train_encoded.drop(columns=['ccs'])
y = train_encoded['ccs']

# Asegurar que X sea un DataFrame con columnas como cadenas
X.columns = X.columns.astype(str)

# División de los datos
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Rellenar valores nulos con la media
X_train = X_train.fillna(X_train.mean())
X_val = X_val.fillna(X_val.mean())

# Seleccionar las 100 mejores características
selector = SelectKBest(score_func=f_regression, k=100)
X_train = selector.fit_transform(X_train, y_train)
X_val = selector.transform(X_val)

# Entrenar el modelo
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predicción y cálculo de MEDAE
y_pred = model.predict(X_val)
medae = median_absolute_error(y_val, y_pred)
print("MEDAE:", medae)



NameError: name 'preprocessor' is not defined

## GENERACIÓN DE PREDICCIONES

In [8]:
# Paso 3: Generación de predicciones

# Codificar la columna "adduct" en el conjunto de prueba con el mismo encoder
adduct_test_encoded = encoder.transform(test[['adduct']])
adduct_test_encoded = pd.DataFrame(adduct_test_encoded, columns=encoder.get_feature_names_out(['adduct']))

# Combinar el conjunto de prueba con las columnas codificadas
test_encoded = pd.concat([test.drop(columns=['adduct']), adduct_test_encoded], axis=1)

# Alinear las columnas de test_encoded con las de X_train
test_encoded = test_encoded.reindex(columns=X.columns, fill_value=0)

# Rellenar valores nulos
test_encoded = test_encoded.fillna(test_encoded.mean())

# Normalizar las características del conjunto de prueba con el mismo escalador
X_test = scaler.transform(test_encoded)

# Seleccionar las mismas 100 mejores características
X_test = selector.transform(X_test)

# Generar predicciones en el conjunto de prueba
test_preds = model.predict(X_test)

# Guardar las predicciones en un archivo CSV
pd.DataFrame(test_preds).to_csv("test_preds.csv", index=False, header=False)




c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Usuario\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SelectKBest was fitted with feature names
  warnings.warn(


## CONCLUSIONES 